## Use the TVM Client Script to get a SigV4 AWS credential

To obtain an AWS SigV4 temporary credential using TVM, use the provided TVM client script and call the `get_sigv4_credentials()` function with an email.

Note: You must provide the values for
- `issuer`: the issuer URL
- `client_id`: the client_id found in SSM Parameter store under the name `/oidc/client_id`
- `client_secret`: the client_secret found in SSM Parameter store under the name `/oidc/client_secret`
- `role_arn`: the IAM role created by the TVM CDK stack to assume, this is the role that has Amazon Q Business permissions
- `region`: the region where the Amazon Q Business Application is setup
- `email`: the user's email

In [ ]:
from tvm_client import TVMClient

token_client = TVMClient(
        issuer="<issuer_url>",
        client_id="<client_id>",
        client_secret="<client_secret>",
        role_arn="<iam_role_arn>",
        region="<region>"
)
    
# Get Sigv4 credentials using TVM
credentials = token_client.get_sigv4_credentials(email="<email>")

### Use SigV4 credentials to initialize Amazon Q Business Client

We will then initialize an Amazon Q Business Boto3 (Python) client with the SigV4 credentials obtained using the TVM Client script to make calls to Amazon Q Business APIs (in this case the `ChatSync` API).

See Amazon Q Business [API documentation](https://docs.aws.amazon.com/amazonq/latest/api-reference/API_Operations_QBusiness.html) for more details on the available APIs.

In [ ]:
!pip install boto3 --upgrade

In [ ]:
import boto3

qbiz = boto3.client("qbusiness", region_name="<region>", **credentials)

chat_params = {
    "applicationId": "<q_business_app_id>",
    "userMessage": "<chat_question>"
}
response = qbiz.chat_sync(**chat_params)

print(response['systemMessage'])
print("=========Sources=========")
for source in response['sourceAttributions']:
    print(f'Title: {source["title"]}, URL: {source["url"]}')
        

You can use the TVM client script with an AWS Lambda function to obtain SigV4 credentials and make calls to Amazon Q Business APIs using the said SigV4 credentials.